In [ ]:
""" ▶ tf.keras.applications.vgg16 """
'''
tf.keras.applications 에 포함된 모델들은 사전에 학습된(pre-trained) 가중치를 가진 유명한 딥러닝 모델의 구현체다.
VGG, ResNet, Inception, MobileNet 등의 다양한 유명한 아키텍처를 제공하며, 이들 모델은 대규모 이미지 데이터셋(주로 ImageNet)에서 미리 학습되었다.
모델을 로드할 때 weights 파라미터를 'imagenet'으로 설정하면 사전에 학습된 가중치를 사용하여 모델을 로드할 수 있다. 가중치를 로드하지 않으려면 weights=None으로 설정하면 된다.

그 중, vgg16을 가져와서 실습해 볼 예정이다.
'''

In [ ]:
""" ▶ 모델 객체, 레이어 속성 """
'''
 1. 이름으로 레이어 객체 리턴: model.get_layer(name, index).name
 2. 인덱스로 레이어 객체 리턴: model.get_layer(name, index) ;  model.layers[index].name ;  model.layers
 3. 레이어 객체의 속성 및 메소드: tf.keras.layers.Layer의 하위 클래스들을 확인
 4. 조건을 만족하는 레이어 오브젝트 리턴, 설정 값 변경
 5. 모델이나 레이어의 가중치(커널의 가중치)나 바이어스 등의 파라미터 값을 리턴
     5-1. 레이어 파라미터(가중치, 바이어스): get_weights(), .weights 속성, trainable_weights, non_trainable_weights, kernel, bias 속성
     5-2. 모델: get_weights(), weights 속성
     5-3. CNN 필터(커널)의 가중치 및 시각화
     
 * 전이학습을 통해서 사용자 모델을 만들고 콜백 핸들러 함수 및 객체를 통해서 튜닝을 할 수 있다.
 CNN, RNN, LSTM_자연어, AE -> (GAN, UNET)
 
 
 tf.keras.applications.vgg16.VGG16(
    include_top=True,
    weights='imagenet',
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation='softmax'
)
'''

In [2]:
# 1. 학습된 모델인 VGG16 가져오기

import tensorflow as tf
from tensorflow.keras.applications import VGG16

model = VGG16()#( weights = None )
model.summary()




Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0  

In [11]:
# 1-2.  모델의 name이나 인덱스로 레이어 오브젝트를  get_layer()로 리턴  
res1 = model.get_layer('block4_conv1')
print(res, res.name)

res2 = model.get_layer(index=11)
print(res, res.name)
print(res1 is res2)

print(model.get_layer(index = -12).name)  # 음수 인덱스 가능.

<keras.src.layers.convolutional.conv2d.Conv2D object at 0x000001E6D514C750> block4_conv1
<keras.src.layers.convolutional.conv2d.Conv2D object at 0x000001E6D514C750> block4_conv1
True
block4_conv1


In [28]:
# 1-3.  모델이 가진 레이어 객체들을 보자. model.layers
print(*model.layers, sep='\n', end='\n\n')

print(type(model.layers))   # model.layers가 list이기 때문에, 아래처럼 인덱싱 가능.
print(model.layers[11])
print(model.layers[-1].name)


<class 'list'>
predictions


In [41]:
# 1-4. 특정 레이어만 추출하기
my_pools = model.layers[-5:]
my_pools = [layer for layer in model.layers if 'pool' in layer.name]
my_pools = [layer for layer in model.layers if isinstance(layer, tf.keras.layers.MaxPooling2D)]
my_pools = [layer for layer in model.layers if isinstance(layer, (tf.keras.layers.MaxPooling2D, tf.keras.layers.Dense))]
print(*my_pools, sep='\n')

In [43]:
# 2. 레이어 객체의 속성 및 메소드를 확인  
# 2-1. 레이어 인스턴스 확인
layer01 = model.get_layer(index =1)
print(layer01)
print(isinstance(layer01, tf.keras.layers.Layer ))
print(isinstance(layer01, tf.keras.layers.Conv2D ) , end='\n\n')

# 2-2. 파라미터 수 리턴  count_params()
print(layer01.count_params())

# 2-3. trainable 속성 바꾸기
' 학습이 끝난 모델을 이용한 전이 학습에서 원하는 레이어만 학습 유무를 지정할 때, 사용함. '
print(layer01.trainable)
layer01.trainable = False
print(layer01.trainable     , end='\n\n')

# 2-4. 원하는 레이어들만 trainable 속성 바꾸기
for layer in model.layers:
    if isinstance(layer, tf.keras.layers.Conv2D):
        layer.trainable = False
print(model.layers[2].trainable)

# 2-5. 레이어 이름 변경도 가능. (한데 할 이유가?)
layer01._name = 'new_name_layer01'
print(layer01.name)

True
True

1792
False
False

False
new_name_layer01


In [44]:
# 3. 임의의모델을 선언하자    Conv2D  ->Flatten -> Dense -> Dense -> BatchNormalization  
model  = tf.keras.Sequential ([ 
    tf.keras.layers.Conv2D (1,(3,3), padding='same', name='L0_conv2d' ,  input_shape=(10,10,1)),
    tf.keras.layers.Flatten ( name='L1_flatten' ),
    tf.keras.layers.Dense ( 10 , name='L2_dense' , use_bias = False ) , 
    tf.keras.layers.Dense ( 1 , name='L3_dense' ) ,
    tf.keras.layers.BatchNormalization()])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 L0_conv2d (Conv2D)          (None, 10, 10, 1)         10        
                                                                 
 L1_flatten (Flatten)        (None, 100)               0         
                                                                 
 L2_dense (Dense)            (None, 10)                1000      
                                                                 
 L3_dense (Dense)            (None, 1)                 11        
                                                                 
 batch_normalization (Batch  (None, 1)                 4         
 Normalization)                                                  
                                                                 
Total params: 1025 (4.00 KB)
Trainable params: 1023 (4.00 KB)
Non-trainable params: 2 (8.00 Byte)
________________________

In [51]:
# 3-1. 모델의 레이어 파라미터(가중치, 바이어스) 리턴을 받자      get_weights()    
layer02 = model.layers[3]
print( type(layer02.get_weights()) , len(layer02.get_weights()))
print(layer02.get_weights(),      end='\n\n')   # get_weights()를 해본 결과 [커널의 가중치, 바이어스 값]이 리턴됨.  -> 현재 가중치는 무작위 초기값을 가진다. 
print('가중치  :', layer02.get_weights()[0].shape, '\n', layer02.get_weights()[0] )
print('바이어스 :', layer02.get_weights()[1] )
# 바로 위에서  L3_dense (Dense) 레이어는 파라미터 11개로 나왔다. 이는 10개의 weight와 1개의 bias 파라미터들의 합이었음을 확인 가능하다.

<class 'list'> 2
[array([[ 0.6960506 ],
       [-0.39610294],
       [ 0.43388206],
       [ 0.12455618],
       [ 0.38311416],
       [ 0.15438235],
       [ 0.7262543 ],
       [ 0.6164519 ],
       [ 0.2885961 ],
       [ 0.52117056]], dtype=float32), array([0.], dtype=float32)]

가중치  : (10, 1) 
 [[ 0.6960506 ]
 [-0.39610294]
 [ 0.43388206]
 [ 0.12455618]
 [ 0.38311416]
 [ 0.15438235]
 [ 0.7262543 ]
 [ 0.6164519 ]
 [ 0.2885961 ]
 [ 0.52117056]]
바이어스 : [0.]


In [64]:
# 3-2. layer02.weights   -> list -> 각 속성은  ResourceVariable 리턴  <- tf.Variable name, shape속성등을 가진다.  
print(layer02.weights[0])
print(type(layer02.weights[0]) ) # ResourceVariable
print('---------------------')
print(layer02.weights[0].name ) 
print(layer02.weights[0].shape) 
print(layer02.weights[1].name ) 
print(layer02.weights[1].shape) 
print('---------------------')
print(layer02.weights[0] is layer02.kernel)    # .kernel과 같다.
print(layer02.weights[1] is layer02.bias)      # .bias와 같다.

<tf.Variable 'L3_dense/kernel:0' shape=(10, 1) dtype=float32, numpy=
array([[ 0.6960506 ],
       [-0.39610294],
       [ 0.43388206],
       [ 0.12455618],
       [ 0.38311416],
       [ 0.15438235],
       [ 0.7262543 ],
       [ 0.6164519 ],
       [ 0.2885961 ],
       [ 0.52117056]], dtype=float32)>
<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'>
---------------------
L3_dense/kernel:0
(10, 1)
L3_dense/bias:0
(1,)
---------------------
True
True


In [53]:
# 3-3 전체 레이어의 get_weights()  가 리턴하는 요소 수를 출력 해보자.  
for layer in model.layers:
    print(f'{layer.name } \t {len(layer.get_weights())}')
# 각 레이어마다 다른 것을 확인할 수 있다.

L0_conv2d 	 2
L1_flatten 	 0
L2_dense 	 1
L3_dense 	 2
batch_normalization 	 4


In [57]:
# 3-4. weights속성을 이용해서 각 레이어의 커널과 bias를 출력
for layer in model.layers:
    print('============')
    for weight in layer.weights:
        print(f"{weight.name} \t {weight.shape}")
len(model.layers)

L0_conv2d/kernel:0 	 (3, 3, 1, 1)
L0_conv2d/bias:0 	 (1,)
L2_dense/kernel:0 	 (100, 10)
L3_dense/kernel:0 	 (10, 1)
L3_dense/bias:0 	 (1,)
batch_normalization/gamma:0 	 (1,)
batch_normalization/beta:0 	 (1,)
batch_normalization/moving_mean:0 	 (1,)
batch_normalization/moving_variance:0 	 (1,)


5

In [62]:
# 4. BatchNormalization()는 trainable=True 해도 non_trainable_weights 해당하는 파라미터가 존재한다.

print(model.layers[4].name, model.layers[4].trainable   ,sep='\t')

for weight in model.layers[4].trainable_weights:
    print(weight.name , "\t", weight.shape)

for weight in model.layers[4].non_trainable_weights:
    print(weight.name , "\t", weight.shape)

batch_normalization	True
batch_normalization/gamma:0 	 (1,)
batch_normalization/beta:0 	 (1,)
batch_normalization/moving_mean:0 	 (1,)
batch_normalization/moving_variance:0 	 (1,)


In [68]:
# 5. 중첩 모델. 모든 모델은 레이어의 하위 클래스이기 때문에, 모델을 하나의 레이어로 선언할 수 있다. 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

inner_model = Sequential([ 
    Dense(100, name='L_in_00', input_shape=(1000, )),
    Dense(10, name='L_in_01')
]) 

outer_model = Sequential( [
    inner_model,
    Dense(1, name='L_out_01')     
]) 


outer_model.summary()
outer_model.layers[0].summary()
print()
print(len (outer_model.weights) , len (outer_model.get_weights()) ) # 6, 6 

for weight in outer_model.weights:
    print(weight.name, weight.shape   ,sep='  \t')

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_7 (Sequential)   (None, 10)                101110    
                                                                 
 L_out_01 (Dense)            (None, 1)                 11        
                                                                 
Total params: 101121 (395.00 KB)
Trainable params: 101121 (395.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 L_in_00 (Dense)             (None, 100)               100100    
                                                                 
 L_in_01 (Dense)             (None, 10)                1010      
                                                          

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pprint

from tensorflow.keras.applications import ResNet50

from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical


from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

In [ ]:
#1 데이터 로드  
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

#2 데이터 축소   : [ : 1000] 학습용 데이터 1000,  테스트 500개  
train_images = train_images[:1000] 
test_images = test_images[:500]
train_labels = to_categorical(train_labels[:1000], 10)
test_labels = to_categorical(test_labels[:500], 10)

#3  ResNet50  모델 사용, 전체  레이어 동결
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

for layer in base_model.layers:
    layer.trainable = False
    
#4. 분류기 추가. 
model = Sequential([
    base_model,
    Flatten(),
    Dense(512, activation='relu'),
    Dense(10, activation='softmax')
])

#model.summary()
#5. 모델 컴파일 
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

#6. 학습  batch_size  = 32, epochs= 3 
model.fit(train_images, train_labels, batch_size=32, epochs=3, validation_split=0.1)

#7. 평가  
loss, accuracy = model.evaluate(test_images, test_labels)
print(f'Test loss: {loss}, Test accuracy: {accuracy}')
